In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from ipywidgets import IntProgress
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


###  Preprocessing the dataset 

In [2]:
# load a dataset from tensorflow datasets
# as_supervised=True will load the dataset in 2-tuple structure [input, target]
# with_info= True provides a tuple containing info about the version, features, # samples of the dataset
mnist_dataset, mnist_info = tfds.load(name='mnist', as_supervised=True, with_info= True)


In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples =  tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image = image / 255.
    return image, label
# map applies a custom transformation to a given dataset
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

# suffling : keeping the same information but in different order

# when we are dealing with enormous dataset, we can't suffle all data at once
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

# the mnist dataset is iterable and in two-tuple format(as_supervized = TRUE)
# iter() creates an object which can be iterated one element at a time (in a for loop)
# next() loads the next element of an iterable object
validation_inputs, validation_targets = next(iter(validation_data))

### Model 

In [4]:
input_size = 784
output_size = 10
hidden_layer_size = 50
# Flatten: transform a tensor to a vector : (28,28,1) -> (784,1)
# Dense: take inputs provided to the model and calculate the dot product of the inputs and the weights
# and adds the biases. this is also where we can apply an activation fuction (it takes the output size)
model = tf.keras.Sequential([
                                tf.keras.layers.Flatten(input_shape=(28,28,1)),
                                tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                                tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                                tf.keras.layers.Dense(output_size, activation='softmax')
                            ])


### chose the optimizer and the loss function

In [5]:
# compile configures the model for training 
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### traning

In [6]:
NUM_EPOCHS = 5

model.fit(train_data, epochs =NUM_EPOCHS, validation_data=(validation_inputs, validation_targets),validation_steps=10, verbose=2)

Epoch 1/5
540/540 - 14s - loss: 0.4163 - accuracy: 0.8845 - val_loss: 0.2082 - val_accuracy: 0.9420
Epoch 2/5
540/540 - 13s - loss: 0.1893 - accuracy: 0.9441 - val_loss: 0.1569 - val_accuracy: 0.9557
Epoch 3/5
540/540 - 14s - loss: 0.1441 - accuracy: 0.9568 - val_loss: 0.1360 - val_accuracy: 0.9615
Epoch 4/5
540/540 - 14s - loss: 0.1177 - accuracy: 0.9650 - val_loss: 0.1183 - val_accuracy: 0.9650
Epoch 5/5
540/540 - 13s - loss: 0.1005 - accuracy: 0.9696 - val_loss: 0.0962 - val_accuracy: 0.9727


### testing the model

In [7]:
# returns the loss value and the metrics values for the model in the test mode
test_loss, test_accuracy = model.evaluate(test_data)
print('test loss : {0:.2f}. Test accuracy : {1: .2f}%'.format(test_loss, test_accuracy*100.))

      1/Unknown - 2s 2s/step - loss: 0.1056 - accuracy: 0.9675test loss : 0.11. Test accuracy :  96.75%
